# Regression

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# download California housing dataset
X, y = fetch_california_housing(return_X_y=True)

# split dataset into training set and testing set (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# do scaling in data (important for regression)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("Shape of training data:", X_train.shape)
print("Shape of testing data:", X_test.shape)

Shape of training data: (14448, 8)
Shape of testing data: (6192, 8)


In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# init random forest model 
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)

# evaluate model without tuning
y_pred = rf.predict(X_test)
initial_mse = mean_squared_error(y_test, y_pred)
print(f"Initial MSE on test set: (without tuning): {initial_mse:.2f}")

Initial MSE on test set: (without tuning): 0.26


In [3]:
import time
from sklearn.model_selection import GridSearchCV


start_time = time.time()
param_grid = {
  'n_estimators': [100, 200, 300],
  'max_depth': [10, 20, 30],
  'min_samples_split': [2, 5, 10],
  'min_samples_leaf': [1, 2, 4],
  'bootstrap': [True, False]
}

# init GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# the best hyperparameters output
print('Best parameters (Grid Search): {grid_search.best_params_}')
best_rf_grid = grid_search.best_estimator_

# evaluate model preformance after grid search
y_pred_grid = best_rf_grid.predict(X_test)
grid_search_mse = mean_squared_error(y_test, y_pred_grid)
print(f'MSE after Grid Search: {grid_search_mse:.2f}')
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.4f} seconds")

Fitting 3 folds for each of 162 candidates, totalling 486 fits
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   8.4s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   8.4s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   8.6s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=  16.9s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=  17.0s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=  16.8s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   8.6s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   8.4s
[

KeyboardInterrupt: 

In [5]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

start_time = time.time()

# define search room for random search
param_dist = {
  'n_estimators': np.arange(100, 500, 100),
  'max_depth': [None] + list(np.arange(10, 50, 10)),
  'min_samples_split': np.arange(2, 11, 2),
  'min_samples_leaf': np.arange(2, 11, 2),
  'bootstrap': [True, False]
}

# init RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=5, cv=3, n_jobs=-1, verbose=2, random_state=42)
random_search.fit(X_train, y_train)

# the best hyperparameters output
print('Best parameters (Random Search): {random_search.best_params_}')
best_rf_random = random_search.best_estimator_

# evaluate model performance after random search
y_pred_random = best_rf_random.predict(X_test)
random_search_mse = mean_squared_error(y_test, y_pred_random)
print(f'MSE after Random Search: {random_search_mse:.2f}')
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.4f} seconds")

Fitting 3 folds for each of 5 candidates, totalling 15 fits
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=8, min_samples_split=2, n_estimators=100; total time=  13.9s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=8, min_samples_split=2, n_estimators=100; total time=  14.5s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=  24.8s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=  24.9s
[CV] END bootstrap=True, max_depth=10, min_samples_leaf=2, min_samples_split=2, n_estimators=300; total time=  25.1s
[CV] END bootstrap=False, max_depth=30, min_samples_leaf=8, min_samples_split=2, n_estimators=100; total time=  14.8s


KeyboardInterrupt: 

In [4]:
from skopt import BayesSearchCV

start_time = time.time()

# define search room for bayesian optimization
param_space = {
  'n_estimators': (100, 500),
  'max_depth': (10, 50),
  'min_samples_split': (2, 11),
  'min_samples_leaf': (2, 11),
  'bootstrap': [True, False]
}

# init BayesSearchCV
bayes_search = BayesSearchCV(estimator=rf, search_spaces=param_space, n_iter=5, cv=3, n_jobs=-1, verbose=2)
bayes_search.fit(X_train, y_train)

# the best hyperparameters output
print('Best parameters (Bayesian Optimization): {bayes_search.best_params_}')
best_rf_bayes = bayes_search.best_estimator_

# evaluate model performance after bayesian optimization
y_pred_bayes = best_rf_bayes.predict(X_test)
bayes_search_mse = mean_squared_error(y_test, y_pred_bayes)
print(f'MSE after Bayesian Optimization: {bayes_search_mse:.2f}')
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.4f} seconds")

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END bootstrap=True, max_depth=35, min_samples_leaf=4, min_samples_split=2, n_estimators=381; total time=  24.4s
[CV] END bootstrap=True, max_depth=35, min_samples_leaf=4, min_samples_split=2, n_estimators=381; total time=  24.6s
[CV] END bootstrap=True, max_depth=35, min_samples_leaf=4, min_samples_split=2, n_estimators=381; total time=  24.9s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END bootstrap=True, max_depth=29, min_samples_leaf=8, min_samples_split=8, n_estimators=463; total time=  26.4s
[CV] END bootstrap=True, max_depth=29, min_samples_leaf=8, min_samples_split=8, n_estimators=463; total time=  26.5s
[CV] END bootstrap=True, max_depth=29, min_samples_leaf=8, min_samples_split=8, n_estimators=463; total time=  26.3s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END bootstrap=True, max_depth=33, min_samples_leaf=3, min_samples_split=4, n_estimators=346; total time=  24.9s
[CV]

# Classification

In [6]:
from sklearn.datasets import fetch_openml
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

X, y = fetch_openml(name='credit-g', version=1, return_X_y=True, as_frame=True)

# convert target to numeric
le = LabelEncoder()
y = le.fit_transform(y)

# do one-hot encoding for categorical features
X_encoded = pd.get_dummies(X, drop_first=True)

# split dataset into training set and testing set (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42)

print("Shape of training data:", X_train.shape)
print("Shape of testing data:", X_test.shape)

Shape of training data: (700, 48)
Shape of testing data: (300, 48)


In [7]:
from sklearn.ensemble import RandomForestClassifier

# init model random forest without hyperparameter tuning
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# evaluate model without tuning
initial_score = rf.score(X_test, y_test)
print(f'Initial accuracy on test set (without tuning): {initial_score:.2f}')

Initial accuracy on test set (without tuning): 0.77


In [8]:
from sklearn.model_selection import GridSearchCV

# define parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy']
}

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

print('Best parameters (Grid Search): {grid_search.best_params_}')
best_rf_grid = grid_search.best_estimator_

# evaluate model performance after grid search
grid_search_score = best_rf_grid.score(X_test, y_test)
print(f'Accuracy after Grid Search: {grid_search_score:.2f}')

Fitting 3 folds for each of 54 candidates, totalling 162 fits
[CV] END criterion=gini, max_depth=10, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END criterion=gini, max_depth=10, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END criterion=gini, max_depth=10, min_samples_split=2, n_estimators=100; total time=   0.3s
[CV] END criterion=gini, max_depth=10, min_samples_split=2, n_estimators=200; total time=   0.8s
[CV] END criterion=gini, max_depth=10, min_samples_split=2, n_estimators=200; total time=   0.8s
[CV] END criterion=gini, max_depth=10, min_samples_split=2, n_estimators=200; total time=   0.8s
[CV] END criterion=gini, max_depth=10, min_samples_split=5, n_estimators=100; total time=   0.5s
[CV] END criterion=gini, max_depth=10, min_samples_split=5, n_estimators=100; total time=   0.5s
[CV] END criterion=gini, max_depth=10, min_samples_split=5, n_estimators=100; total time=   0.4s
[CV] END criterion=gini, max_depth=10, min_samples_split=2, n_est

In [9]:
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_dist = {
    'n_estimators': np.linspace(100, 500, 100, dtype=int),
    'max_depth': np.linspace(10, 50, 5, dtype=int),
    'min_samples_split': [2, 5, 10],
    'criterion': ['gini', 'entropy']
}

random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=20, cv=3, n_jobs=-1, verbose=2, random_state=42)
random_search.fit(X_train, y_train)

print('Best parameters (Random Search): {random_search.best_params_}')
best_rf_random = random_search.best_estimator_

random_search_score = best_rf_random.score(X_test, y_test)
print(f'Accuracy after Random Search: {random_search_score:.2f}')

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] END criterion=gini, max_depth=40, min_samples_split=10, n_estimators=221; total time=   0.4s
[CV] END criterion=gini, max_depth=30, min_samples_split=10, n_estimators=342; total time=   0.7s
[CV] END criterion=gini, max_depth=30, min_samples_split=10, n_estimators=342; total time=   0.7s
[CV] END criterion=gini, max_depth=40, min_samples_split=10, n_estimators=221; total time=   0.5s
[CV] END criterion=gini, max_depth=30, min_samples_split=10, n_estimators=342; total time=   0.7s
[CV] END criterion=gini, max_depth=40, min_samples_split=10, n_estimators=221; total time=   0.7s
[CV] END criterion=gini, max_depth=50, min_samples_split=2, n_estimators=479; total time=   1.3s
[CV] END criterion=gini, max_depth=50, min_samples_split=2, n_estimators=479; total time=   1.3s
[CV] END criterion=gini, max_depth=50, min_samples_split=2, n_estimators=479; total time=   1.3s
[CV] END criterion=entropy, max_depth=10, min_samples_split=

In [10]:
from skopt import BayesSearchCV

param_space = {
    'n_estimators': (100, 500),
    'max_depth': (10, 50),
    'min_samples_split': (2, 10),
    'criterion': ['gini', 'entropy']
}

bayes_search = BayesSearchCV(estimator=rf, search_spaces=param_space, n_iter=32, cv=3, n_jobs=-1, verbose=2, random_state=42)
bayes_search.fit(X_train, y_train)

print(f'Best parameters (Bayesian Optimization): {bayes_search.best_params_}')
best_rf_bayes = bayes_search.best_estimator_

bayes_search_score = best_rf_bayes.score(X_test, y_test)
print(f'Accuracy after Bayesian Optimization: {bayes_search_score:.2f}')

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END criterion=gini, max_depth=39, min_samples_split=9, n_estimators=226; total time=   0.3s
[CV] END criterion=gini, max_depth=39, min_samples_split=9, n_estimators=226; total time=   0.4s
[CV] END criterion=gini, max_depth=39, min_samples_split=9, n_estimators=226; total time=   0.4s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END criterion=entropy, max_depth=45, min_samples_split=4, n_estimators=480; total time=   0.9s
[CV] END criterion=entropy, max_depth=45, min_samples_split=4, n_estimators=480; total time=   0.9s
[CV] END criterion=entropy, max_depth=45, min_samples_split=4, n_estimators=480; total time=   0.9s
Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END criterion=gini, max_depth=47, min_samples_split=3, n_estimators=273; total time=   0.4s
[CV] END criterion=gini, max_depth=47, min_samples_split=3, n_estimators=273; total time=   0.4s
[CV] END criterion=gini, max_depth=47,